In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb

Using Theano backend.


In [11]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features,
                                                      test_split=0.2)

Loading data...


In [12]:
type(X_train)

numpy.ndarray

In [13]:
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

20000 train sequences
5000 test sequences


In [14]:
X_train.shape

(20000,)

In [16]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
X_train shape: (20000, 80)
X_test shape: (5000, 80)


In [17]:
X_train

array([[ 360,    7,   19, ...,  256,   82,    7],
       [   6,  102,   37, ...,    5, 1324,    7],
       [ 853,    6,   51, ...,   28, 1872,   43],
       ..., 
       [  19, 1174,   17, ...,  546, 2244,    7],
       [  10, 1972,   49, ...,   13, 1053,    7],
       [  69,  138,    7, ...,   51,   91,    7]], dtype=int32)

In [18]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

Build model...


In [19]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
print('Train...')
print(X_train.shape)
print(y_train.shape)

Train...
(20000, 80)
(20000,)


In [21]:
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test))

Train on 20000 samples, validate on 5000 samples
Epoch 1/15
20000/20000 [==============================] - 135s - loss: 0.5731 - acc: 0.6960 - val_loss: 0.4340 - val_acc: 0.8030
Epoch 2/15
20000/20000 [==============================] - 141s - loss: 0.4221 - acc: 0.8132 - val_loss: 0.3903 - val_acc: 0.8226
Epoch 3/15
20000/20000 [==============================] - 142s - loss: 0.3353 - acc: 0.8570 - val_loss: 0.3857 - val_acc: 0.8238
Epoch 4/15
20000/20000 [==============================] - 141s - loss: 0.2772 - acc: 0.8864 - val_loss: 0.3912 - val_acc: 0.8238
Epoch 5/15
20000/20000 [==============================] - 148s - loss: 0.2304 - acc: 0.9091 - val_loss: 0.4525 - val_acc: 0.8170
Epoch 6/15
20000/20000 [==============================] - 146s - loss: 0.1919 - acc: 0.9259 - val_loss: 0.5348 - val_acc: 0.8100
Epoch 7/15
20000/20000 [==============================] - 150s - loss: 0.1634 - acc: 0.9388 - val_loss: 0.4766 - val_acc: 0.8160
Epoch 8/15
20000/20000 [========================

In [22]:
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

5000/5000 [==============================] - 8s     
Test score: 0.668048014212
Test accuracy: 0.8154


In [2]:
import itertools
import numpy as np

In [3]:
sentences = '''
sam is red
hannah not red
hannah is green
bob is green
bob not red
sam not green
sarah is red
sarah not green'''.strip().split('\n')

In [4]:
sentences

['sam is red',
 'hannah not red',
 'hannah is green',
 'bob is green',
 'bob not red',
 'sam not green',
 'sarah is red',
 'sarah not green']

In [5]:
is_green = np.asarray([[0, 1, 1, 1, 1, 0, 0, 0]], dtype='int32').T

In [6]:
is_green

array([[0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0]], dtype=int32)

In [7]:
lemma = lambda x: x.strip().lower().split(' ')
sentences_lemmatized = [lemma(sentence) for sentence in sentences]

In [8]:
sentences_lemmatized

[['sam', 'is', 'red'],
 ['hannah', 'not', 'red'],
 ['hannah', 'is', 'green'],
 ['bob', 'is', 'green'],
 ['bob', 'not', 'red'],
 ['sam', 'not', 'green'],
 ['sarah', 'is', 'red'],
 ['sarah', 'not', 'green']]

In [9]:
type(sentences_lemmatized)

list

In [10]:
words = set(itertools.chain(*sentences_lemmatized))

In [11]:
words

{'bob', 'green', 'hannah', 'is', 'not', 'red', 'sam', 'sarah'}

In [12]:
# dictionaries for converting words to integers and vice versa
word2idx = dict((v, i) for i, v in enumerate(words))
idx2word = list(words)

In [13]:
word2idx

{'bob': 6,
 'green': 4,
 'hannah': 2,
 'is': 3,
 'not': 5,
 'red': 7,
 'sam': 1,
 'sarah': 0}

In [14]:
idx2word

['sarah', 'sam', 'hannah', 'is', 'green', 'not', 'bob', 'red']

In [15]:
# convert the sentences a numpy array
to_idx = lambda x: [word2idx[word] for word in x]
sentences_idx = [to_idx(sentence) for sentence in sentences_lemmatized]
sentences_array = np.asarray(sentences_idx, dtype='int32')

In [16]:
sentences_array

array([[1, 3, 7],
       [2, 5, 7],
       [2, 3, 4],
       [6, 3, 4],
       [6, 5, 7],
       [1, 5, 4],
       [0, 3, 7],
       [0, 5, 4]], dtype=int32)

In [17]:
# parameters for the model
sentence_maxlen = 3
n_words = len(words)
n_embed_dims = 3

In [18]:
# put together a model to predict 
from keras.layers import Input, Embedding, merge, Flatten, SimpleRNN
from keras.models import Model

Using Theano backend.


In [19]:
input_sentence = Input(shape=(sentence_maxlen,), dtype='int32')
input_embedding = Embedding(n_words, n_embed_dims)(input_sentence)
color_prediction = SimpleRNN(1)(input_embedding)

In [20]:
predict_green = Model(input=[input_sentence], output=[color_prediction])
predict_green.compile(optimizer='sgd', loss='binary_crossentropy')

In [21]:
# fit the model to predict what color each person is
predict_green.fit([sentences_array], [is_green], nb_epoch=500, verbose=1)

Epoch 1/500
8/8 [==============================] - 0s - loss: 5.1168
Epoch 2/500
8/8 [==============================] - 0s - loss: 0.6719
Epoch 3/500
8/8 [==============================] - 0s - loss: 0.6579
Epoch 4/500
8/8 [==============================] - 0s - loss: 0.6453
Epoch 5/500
8/8 [==============================] - 0s - loss: 0.6336
Epoch 6/500
8/8 [==============================] - 0s - loss: 0.6226
Epoch 7/500
8/8 [==============================] - 0s - loss: 0.6120
Epoch 8/500
8/8 [==============================] - 0s - loss: 0.6018
Epoch 9/500
8/8 [==============================] - 0s - loss: 0.5919
Epoch 10/500
8/8 [==============================] - 0s - loss: 0.5822
Epoch 11/500
8/8 [==============================] - 0s - loss: 0.5726
Epoch 12/500
8/8 [==============================] - 0s - loss: 0.5632
Epoch 13/500
8/8 [==============================] - 0s - loss: 0.5538
Epoch 14/500
8/8 [==============================] - 0s - loss: 0.5444
Epoch 15/500
8/8 [===========

In [22]:
embeddings = predict_green.layers[1].W.get_value()

In [23]:
embeddings

array([[-0.19183166,  0.28765371, -0.15641065],
       [-0.12936352,  0.28779608, -0.20064083],
       [ 0.19828902, -0.30468383,  0.2721456 ],
       [-0.07414302,  0.13606086, -0.12131353],
       [ 0.09109619, -0.21298023,  0.12953848],
       [-0.02834477,  0.11797626, -0.09546226],
       [ 0.18700415, -0.34668186,  0.30039063],
       [ 0.07209901, -0.15533428,  0.09490708]], dtype=float32)